In [1]:
import numpy as np
import Utilityfunctions as utils
import jax.numpy as jnp
import ssr_kronvec_jax as ssr
import ssr_likelihood_jax as jax_lik
import ssr_likelihood as pyt

In [2]:
n = 3
theta = utils.random_theta(n, 0.0)
n_dat = 100
dat = np.random.binomial(n=1, p=0.6, size=n_dat*(2*n+1))
dat = jnp.array(dat).reshape((n_dat, -1))
dat = dat.at[:,-1].set(1)
dat = jnp.column_stack((dat, jnp.ones(n_dat, dtype=int)))
lam1 = 0.5
lam2 = 1.5

In [3]:
#dat_point = dat.at[0,0:2*n+1].get()
#m = dat_point.sum()
#p0 = np.zeros(2**m)
#p0[0] = 1
#pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)

In [4]:
#dat_point = jnp.array([1,1,0,0,1,1,1], dtype = int)
#m = dat_point.sum()
#p0 = np.zeros(2**m)
#p0[0] = 1
#pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)
#count = dat_point[1:2*n:2].sum()
#latent_dist = jnp.zeros(2**(count))
#print(utils.ssr_obs_dist(pTh1, dat_point, n, obs_prim=True))
#inds = ssr.obs_inds(pTh1, dat_point, latent_dist, obs_prim=True)
#utils.trunk_states(dat_point)[2**(m-1)+inds[0].at[0].get()]

In [5]:
#jax_lik.log_prob_coupled(dat, theta, lam1, lam2)

In [6]:
#jax_lik.log_prob_single(dat, jnp.array(theta), lam1)

In [7]:
#jax_lik.grad_single(dat, jnp.array(theta), lam1)

In [30]:
state = jnp.array([1,0,1,1,1,0,1])#dat.at[2, 0:2*n+1].get()
m = state.sum()
count = state.at[1:2*n+1:2].get().sum()+1
latent_state = state.at[0:2*n:2].get()
print(count)
print(latent_state)
latent_dist = jnp.zeros(2**(count))
p0 = jnp.zeros(2**int(state.sum()))
p0 = p0.at[0].set(1.)
pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = state, transpose = False)

#jax_lik._grad_coupled(theta, lam1, lam2, state, p0, latent_dist, latent_state, True)
inds = ssr.obs_inds(pTh1, state, latent_dist, obs_prim=True)
pTh1_obs = pTh1.at[inds].get()
nk = pTh1_obs.sum()
q = jnp.zeros_like(p0)
pos = 2**(m-1) + inds.at[0].get()
q = q.at[pos].set(1/nk)
#print(q)
print(utils.trunk_states(state)[inds])
print(pTh1_obs.shape)
# Actual gradient
#q = ssr.marg_transp(q, state, marg_met=False, marg_seeding=False)
#print(q)
#q = jax_lik.R_i_inv_vec(theta, q, lam1, state, transpose = True)
#g_1 = jax_lik.x_partial_Q_y(theta, q, pTh1, state)
#g_1

2
[1 1 1]
['1010100' '1011100' '1010101' '1011101']
(4,)


In [31]:
matA = np.array([[1,1,0,0],[0,0,1,1]])
matB = np.array([[1,0],[0,1]])
matC = np.kron(np.kron(np.kron(matB, matA),matB),matB)
#A = utils.ssr_marginalize(iwas.T, n, state, marg_met=True)
B = matC @ pTh1
iwas = np.zeros_like(B)
iwas[-1] = 1/B[-1]
res1 = iwas.T @ matC
#A = utils.ssr_marginalize(iwas, n, state, marg_met=True)
iwas2 = np.zeros_like(pTh1)
iwas2[inds] = 1/B[-1]
print(res1)
print(iwas2)
#print(A)


[   0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.         1316.44616699    0.            0.
    0.         1316.44616699]
[   0.        0.        0.        0.        0.        0.        0.
    0.        0.        0.        0.     1316.4462    0.        0.
    0.     1316.4462    0.        0.        0.        0.        0.
    0.        0.        0.        0.        0.        0.     1316.4462
    0.        0.        0.     1316.4462]
